## Randon forest aplicado a previsão de retornos

Analise de indicadores técnicos na previsão dos retornos da ação da petrobras (PETR4)

In [1]:
### Importando bibliotecas
import pandas as pd
import numpy as np
import ta  ### biblioteca para analise de indicadores técnicos
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
